## Preprocessing

In [1]:
import pandas as pd
from src.preprocessing.load_dataset import load_dataset
from src.preprocessing.pipeline import preprocess
from src.preprocessing.transform_target import transform_target
from src.utils.save import save_processed, save_data_summary, save_splits
from src.preprocessing.split import split_data
from src.preprocessing.encode import encode
from src.preprocessing.scale_data import scale_data

In [2]:
DATASETS = ["ibtracs.last3years"]
for ds in DATASETS:
    df_raw, cfg = load_dataset(ds)
    X_raw, y_raw = preprocess(df_raw.copy(), cfg)
    y = transform_target(y_raw, cfg, ds)
    X = X_raw
    save_processed(X, y, ds)
    save_data_summary(X, y, ds)
    splits = split_data(X, y)

    # Encode: fit on train, transform both
    splits["X_train"], splits["X_test"], encoder = encode(
        splits["X_train"], 
        splits["X_test"]
    )

    # Scale: fit on train, transform both
    splits["X_train"], splits["X_test"], scaler = scale_data(
        splits["X_train"],
        splits["X_test"]
    )

    save_splits(splits, ds, encoder=encoder, scaler=scaler)

/Users/anitarazafi/Desktop/masters/code/feature-selection/src/preprocessing/normalize_missing_values.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[obj_cols] = df[obj_cols].replace(GENERIC_MISSING, np.nan)



DROPPING HIGH MISSING VALUE COLUMNS (> 60%)
Dropping 143 columns:
  - WMO_WIND                                 : 67.3% missing
  - WMO_PRES                                 : 63.5% missing
  - WMO_AGENCY                               : 63.0% missing
  - USA_RECORD                               : 99.7% missing
  - USA_R50_NE                               : 77.7% missing
  - USA_R50_SE                               : 77.7% missing
  - USA_R50_SW                               : 77.8% missing
  - USA_R50_NW                               : 77.7% missing
  - USA_R64_NE                               : 85.6% missing
  - USA_R64_SE                               : 85.6% missing
  - USA_R64_SW                               : 85.7% missing
  - USA_R64_NW                               : 85.6% missing
  - USA_EYE                                  : 97.6% missing
  - TOKYO_LAT                                : 81.2% missing
  - TOKYO_LON                                : 81.2% missing
  - TOKYO_GRADE   